# Interface for calling a subset of GETTSIM's depiction of the German taxes and transfers system

In many applications, just a subset of functions is required. That is, you
may want to ignore the means-tested transfers component, not think about
child benefits, etc..

The advantage is that computations are more transparent and shorter, and that
the number of required input variables is smaller than for the full-blown
version.

## Two sections:

1. The beginning of this notebook walks you through one particular usecase in five steps
2. The second part explains how to make choices for setting your problem up

# Part 1: Solution to one example

We are interested in taxes and social insurance contributions,
ignoring any tax deductions, for a single individual without kids. 


## Step 1: Set up the required data.

In [1]:
import pandas as pd

In [2]:
input_data = pd.DataFrame(
    {
        "hh_id": 1,
        "tu_id": 1,
        "p_id": 1,
        "bruttolohn_m": 2000,
        "_zu_verst_kapital_eink_tu": 0,
        "selbstständig": False,
        "eink_selbst_m": 0,
        "ges_rente_m": 0,
        "_zu_verst_eink_kein_kinderfreib_tu": 24000,
        "alter": 35,
        "wohnort_ost": False,
        "hat_kinder": False,
        "kind": False,
        "prv_krankenv": False,
        "kinderfreib_tu": 0,
        "_kindergeld_m_tu_basis": 0,
    },
    index=[0],
)
input_data.T

,0
hh_id,1
tu_id,1
p_id,1
bruttolohn_m,2000
_zu_verst_kapital_eink_tu,0
selbstständig,False
eink_selbst_m,0
ges_rente_m,0
_zu_verst_eink_kein_kinderfreib_tu,24000
alter,35


These contain: 
- The first three columns are identifiers for person, tax unit, and household
- The next six columns define the income:
  - `"bruttolohn_m": 2000` defines the monthly gross wage to be 2000€
  - `"_zu_verst_kapital_eink_tu": 0` sets capital income to zero
  - `"selbstständig": False` defines that the individual is not self-employed
  - `"eink_selbst_m": 0` defines the income from self-employment to be zero
  - `"ges_rente_m": 0` sets the individual's pension income to zero
  - `"_zu_verst_eink_kein_kinderfreib_tu": 24000` defines the individual's taxable income to be twelve times his monthly gross wage in order to ignore the machinery calculating deductions
- `"alter"`, `"wohnort_ost"`, `"hat_kinder"`, `"kind"`, `"prv_krankenv"` define variables that are relevant for calculating the social insurance contributions
- `"kinderfreib_tu"`, `"_kindergeld_m_tu_basis"` are a shortcut for setting child benefits and child tax dedutions to zero.

## Step 2: Set up the Taxes and Transfers System

In [3]:
from gettsim import get_policies_for_date


params_dict, policy_func_dict = get_policies_for_date("2020")

To load the parameters for the tax and transfer system, you have to provide a date to
the function `get_policies_for_date` which can be given as a string. 

`"2020"` loads the correct parameters for the year 2020. It is equivalent to `"2020-01-01"`
meaning the state of the tax and transfer system at the first day of 2020.

The function has two return values.

- `params_dict` is a dictionary containing date-specific parameters.
- `policy_func_dict` is a dictionary containing functions which are necessary to compute quantities in the tax and transfer system on the provided date.

# Step 3: Select user-provided variables overriding GETTSIM calculations and targets

GETTSIM is able to compute many quantities internally. If you want GETTSIM to take some variables in the data as given and proceed computations from thereon, you have to pass the variable names as a list.

In [4]:
user_columns = [
    "_kindergeld_m_tu_basis",
    "kinderfreib_tu",
    "_zu_verst_kapital_eink_tu",
    "_zu_verst_eink_kein_kinderfreib_tu",
]

Finally, we have to define the targets or the variables we would like to compute. `targets` can be a string or a list of strings.

If you pass no targets, GETTSIM attempts to compute all possible variables in the tax and transfer system which requires more input data.

In [5]:
targets = [
    "eink_st_tu",
    "soli_st_tu",
    "ges_krankenv_beitr_m",
    "rentenv_beitr_m",
    "arbeitsl_v_beitr_m",
    "rentenv_beitr_m",
]

## Step 4: Call GETTSIM's main function


To compute the target variables, call `compute_taxes_and_transfers` with the correct inputs.

In [6]:
from gettsim import compute_taxes_and_transfers


output_data = compute_taxes_and_transfers(
    data=input_data,
    user_functions=policy_func_dict,
    user_columns=user_columns,
    params=params_dict,
    targets=targets,
)
output_data.round(2).T

,0
eink_st_tu,3432.33
soli_st_tu,188.78
ges_krankenv_beitr_m,157.00
rentenv_beitr_m,186.00
arbeitsl_v_beitr_m,24.00
rentenv_beitr_m,186.00


Often you will want to combine`input_data` and `output_data`, for example, in order to plot
taxes as a function of income. Since the index of both is guaranteed to be the same, this is
simply a matter of calling the `join` method:

In [7]:
data = input_data.join(output_data.round(2))
data.T

,0
hh_id,1
tu_id,1
p_id,1
bruttolohn_m,2000
_zu_verst_kapital_eink_tu,0
selbstständig,False
eink_selbst_m,0
ges_rente_m,0
_zu_verst_eink_kein_kinderfreib_tu,24000
alter,35


# Part 2: How to set up such problems yourself

tbc